<a href="https://colab.research.google.com/github/erikafreeman/Topic1/blob/main/Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def userpath(user):
    if user in 'ecf':
        userspecific = "/4-CareerWork/1-PhD"
        return userspecific 
    if user in 'ajt':
        userspecific = ""
        return userspecific 
    else:
        return (user + ' is is not a valid username. Try again. Initials, no caps.')

In [3]:
#ajt or ecf
user = input("Enter username: ")
user_path =  userpath(user)
general_gdrive = "/content/drive/MyDrive"
print('The user of this session is: ' + user)
print('We will connect you to this google drive location: ' + general_gdrive + user_path)

Enter username: ecf
The user of this session is: ecf
We will connect you to this google drive location: /content/drive/MyDrive/4-CareerWork/1-PhD


In [4]:
#general drive locations 
user_path =  userpath(user)
general_gdrive = "/content/drive/MyDrive"
folder_name = "/1-SoilGradient/"
path = general_gdrive + user_path + folder_name
r_lib = '1-Code/Rlib/library.tar.gz'
figures_to = '2-Figures/2-Figures/'
csv_from = '0-Raw/'
csv_to = '1-Processed/'

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as pl
import matplotlib.pyplot as plt

Mounted at /content/drive


In [6]:
#Magic command interface for interactive work with R in ipython. 
'''
%R and %%R are the line and cell magics, respectively
https://rpy2.github.io/doc/latest/html/interactive.html
'''
%load_ext rpy2.ipython

In [7]:
#Point to zipped R packages(saved in another notebook)
#https://stackoverflow.com/questions/35497069/passing-ipython-variables-as-arguments-to-bash-commands
r_path = '"' + path + r_lib + '"' 
%cp $r_path . 
!tar xf library.tar.gz

tar: Removing leading `/' from member names


In [8]:
#Add packages to path
%%R
.libPaths('usr/local/lib/R/site-library/')

In [9]:
dir = path + '0-Data/1-Processed/'
data_dir = dir +  '01-Cleaned/'
filename = '00-CleanData_original_normal.csv'

In [26]:
df = pd.read_csv(data_dir + filename)
print(df.columns)

Index(['Unnamed: 0', 'formula_isotopefree', 'mz', 'C', 'H', 'O', 'N', 'S', 'P',
       'Cl', 'Na', 'C1B05', 'C1B15', 'C1B30', 'C1B60', 'C1F05', 'C1F15',
       'C1F30', 'C1F60', 'C1S05', 'C1S15', 'C1S30', 'C1S60', 'C1ST', 'C1T05',
       'C1T15', 'C1T30', 'C1T60', 'C2B05', 'C2B15', 'C2B30', 'C2B60', 'C2F05',
       'C2F15', 'C2F30', 'C2F60', 'C2S05', 'C2S15', 'C2S30', 'C2S60', 'C2ST',
       'C2T05', 'C2T15', 'C2T30', 'C2T60', 'H1B05', 'H1B15', 'H1B30', 'H1B60',
       'H1F05', 'H1F15', 'H1F30', 'H1F60', 'H1S05', 'H1S15', 'H1S30', 'H1S60',
       'H1ST', 'H1T05', 'H1T15', 'H1T30', 'H1T60', 'H2B05', 'H2B15', 'H2B30',
       'H2B60', 'H2F05', 'H2F15', 'H2F30', 'H2F60', 'H2S05', 'H2S15', 'H2S30',
       'H2S60', 'H2ST', 'H2T05', 'H2T15', 'H2T30', 'H2T60'],
      dtype='object')


In [71]:
df1 = df.copy()
df1.loc[:,'C1B05':] = df1.loc[:,'C1B05':].apply(lambda x: [1 if y > 0 else 0 for y in x])
SiteNamesList = ['C1', 'C2', 'H1', 'H2']
df1.set_index('formula_isotopefree', inplace=True)
df2 = df1.filter(regex='|'.join(SiteNamesList))
df1_transposed = df2.T 
df3 = df1_transposed.reset_index()
df3.rename(columns={"index": "Sample"}, inplace=True)


def WithSampleNameSplit(df):
    stacked_mol_sample = df['Sample'].str.split('(\d+)([A-Za-z]+)', n = 3, expand = True) 
    stacked_mol_sample['SiteName'] = stacked_mol_sample[0]+ stacked_mol_sample[1] 
    stacked_mol_names = stacked_mol_sample.rename(columns={2: 'Position', 3: 'Depth'})
    mol_final_clean= stacked_mol_names.drop([0, 1], axis=1) #624909 rows × 8 columns
    mol_final_clean['Depth'].replace({"": "ST"}, inplace=True) #replace blanks with ST #mol_final_clean['Depth'].unique()
    with_names = pd.concat([df, mol_final_clean], axis = 1) 
    return with_names

df4 = WithSampleNameSplit(df3)
display(df4)
df4['Depth']= df4['Depth'].astype('category')
df4['Depth']= df4['Depth'].cat.reorder_categories(['05', '15', '30', '60', 'ST'], ordered=True)
df4= df4.sort_values('Depth')

df4['Position']= df4['Position'].astype('category')
df4['Position']= df4['Position'].cat.reorder_categories(['S', 'B', 'F', 'T', 'ST'],  ordered=True)
df4= df4.sort_values('Position')

,Sample,C10H10N2O2,C10H10N2O3,C10H10N2O4,C10H10N2O5,C10H10O,C10H10O10,C10H10O4,C10H10O5,C10H10O5S,...,C9H9NO2,C9H9NO3,C9H9NO4,C9H9NO5,C9H9NO6,C9H9NO7,C9H9O10P,Position,Depth,SiteName
0,C1B05,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,B,05,C1
1,C1B15,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,0,1,B,15,C1
2,C1B30,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,0,0,B,30,C1
3,C1B60,1,1,1,1,1,0,1,1,0,...,1,1,1,1,1,0,0,B,60,C1
4,C1F05,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,F,05,C1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,H2ST,0,1,1,1,1,0,1,1,0,...,1,1,1,1,1,0,0,ST,ST,H2
64,H2T05,0,1,0,1,1,0,1,1,1,...,1,1,1,1,1,0,0,T,05,H2
65,H2T15,0,1,1,1,1,0,1,1,0,...,1,1,1,1,1,1,0,T,15,H2
66,H2T30,1,1,1,1,1,0,1,1,0,...,1,1,1,1,1,1,0,T,30,H2


In [73]:
df5 = df4.groupby(['SiteName', 'Depth']).sum()
df5.loc[:,'C10H10N2O2':] = df5.loc[:,'C10H10N2O2':].apply(lambda x: [1 if y > 0 else 0 for y in x])
df5['sum'] =df5.sum(axis=1)
df5['sum2'] = 1216/df5['sum'] * 100
display(df5)

print(1216/9327)

C10H10N2O2  C10H10N2O3  C10H10N2O4  C10H10N2O5  C10H10O  \
SiteName Depth                                                            
C1       05              1           1           1           1        1   
         15              1           1           1           1        1   
         30              1           1           1           1        1   
         60              1           1           1           1        1   
         ST              1           1           1           1        1   
C2       05              1           1           1           1        1   
         15              1           1           1           1        1   
         30              1           1           1           1        1   
         60              1           1           1           1        1   
         ST              0           1           1           1        1   
H1       05              1           1           1           1        1   
         15              1           1           1           1        1   
         30              1           1           1           1        1   
         60              1           1           1           1        1   
         ST              0           1           1           1        1   
H2       05              1           1           1           1        1   
         15              1           1           1           1        1   
         30              1           1           1           1        1   
         60              1           1           1           1        1   
         ST              0           1           1           1        1   

                C10H10O10  C10H10O4  C10H10O5  C10H10O5S  C10H10O8  ...  \
SiteName Depth                                                      ...   
C1       05             1         1         1          1         1  ...   
         15             1         1         1          0         1  ...   
         30             1         1         1          1         1  ...   
         60             1         1         1          1         1  ...   
         ST             0         1         1          0         1  ...   
C2       05             1         1         1          0         1  ...   
         15             1         1         1          0         1  ...   
         30             1         1         1          1         1  ...   
         60             0         1         1          1         1  ...   
         ST             1         1         1          0         1  ...   
H1       05             1         1         1          1         1  ...   
         15             1         1         1          1         1  ...   
         30             0         1         1          1         1  ...   
         60             1         1         1          1         1  ...   
         ST             0         1         1          0         1  ...   
H2       05             1         1         1          1         1  ...   
         15             1         1         1          0         1  ...   
         30             1         1         1          0         1  ...   
         60             0         1         1          1         1  ...   
         ST             0         1         1          0         1  ...   

                C9H9NO  C9H9NO2  C9H9NO3  C9H9NO4  C9H9NO5  C9H9NO6  C9H9NO7  \
SiteName Depth                                                                 
C1       05          1        1        1        1        1        1        1   
         15          1        1        1        1        1        1        1   
         30          1        1        1        1        1        1        1   
         60          1        1        1        1        1        1        0   
         ST          0        1        1        1        1        1        1   
C2       05          1        1        1        1        1        1        1   
         15          1        1        1        1        1        1       

0.13037418248096924
